<a href="https://colab.research.google.com/github/TillVollmer5/mass_spectroscopy/blob/main/MS_Service_Blank_Subtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive
import pandas as pd

drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [15]:
Blank_EtOAc = pd.read_csv('/content/drive/My Drive/Reusser_alkene_screening/EtOAC.csv', error_bad_lines=False)
Blank_DCM = pd.read_csv('/content/drive/My Drive/Reusser_alkene_screening/DCM.csv', error_bad_lines=False)
ER_0789a_1 = pd.read_csv('/content/drive/My Drive/Reusser_alkene_screening/ER_0789a_1.csv', error_bad_lines=False)
ER_0788a_1 = pd.read_csv('/content/drive/My Drive/Reusser_alkene_screening/ER_0788a_1.csv', error_bad_lines=False)
ER_0787a_1 = pd.read_csv('/content/drive/My Drive/Reusser_alkene_screening/ER_0787a_1.csv', error_bad_lines=False)
ER_0787b_1 = pd.read_csv('/content/drive/My Drive/Reusser_alkene_screening/ER_0787a_1.csv', error_bad_lines=False)

<ipython-input-15-c46f05a3ca8c>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  Blank_EtOAc = pd.read_csv('/content/drive/My Drive/Reusser_alkene_screening/EtOAC.csv', error_bad_lines=False)
<ipython-input-15-c46f05a3ca8c>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  Blank_DCM = pd.read_csv('/content/drive/My Drive/Reusser_alkene_screening/DCM.csv', error_bad_lines=False)
<ipython-input-15-c46f05a3ca8c>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  ER_0789a_1 = pd.read_csv('/content/drive/My Drive/Reusser_alkene_screening/ER_0789a_1.csv', error_bad_lines=False)
<ipython-input-15-c46f05a3ca8c>:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.

In [16]:
Blank_EtOAc.fillna(0, inplace=True)
Blank_DCM.fillna(0, inplace=True)
ER_0789a_1.fillna(0, inplace=True)
ER_0788a_1.fillna(0, inplace=True)
ER_0787a_1.fillna(0, inplace=True)
ER_0787b_1.fillna(0, inplace=True)

In [17]:
print(Blank_EtOAc.shape)
print(Blank_DCM.shape)
print(ER_0789a_1.shape)
print(ER_0788a_1.shape)
print(ER_0787a_1.shape)
print(ER_0787b_1.shape)

(44, 21)
(25, 21)
(147, 21)
(175, 21)
(104, 21)
(104, 21)


In [18]:
def combine_df(df1, df2):
    combined_df = pd.concat([df1, df2], ignore_index=True)
    return combined_df

def exclusion_list(sample_df, blank_df, retention_time_threshold=1):
    sample_df_copy = sample_df.copy()
    blank_df_copy = blank_df.copy()

    exclusion_list = []

    for index_sample, row_sample in sample_df_copy.iterrows():
        for index_blank, row_blank in blank_df_copy.iterrows():
            if row_sample["Formula (mol ion)"] == row_blank["Formula (mol ion)"]:
                if abs(row_sample["Retention Time"] - row_blank["Retention Time"]) > retention_time_threshold:
                    exclusion_list.append(index_sample)
                    break

    return exclusion_list

def remove_rows(df, exclusion_list):
    df_copy = df.copy()

    df_copy = df_copy.drop(exclusion_list)

    return df_copy

def filter_df(df):
    rows_to_remove = []

    for index, row in df.iterrows():
        if row['HRF Score'] < 90 or row['SI'] < 800:
            rows_to_remove.append(index)

    filtered_df = df.drop(rows_to_remove)

    return filtered_df

def rm_no_RI(df):
    rows_to_remove = []

    for index, row in df.iterrows():
        if row['Library RI'] == 0 or row['Calculated RI'] == 0:
            rows_to_remove.append(index)

    filtered_df = df.drop(rows_to_remove)

    return filtered_df

In [19]:
Blank_data_df = combine_df(Blank_EtOAc, Blank_DCM)

ER_0789a_1_el = exclusion_list(ER_0789a_1, Blank_data_df)
ER_0789a_1_rr= remove_rows(ER_0789a_1, ER_0789a_1_el)

ER_0788a_1_el = exclusion_list(ER_0788a_1, Blank_data_df)
ER_0788a_1_rr = remove_rows(ER_0788a_1, ER_0788a_1_el)

ER_0787a_1_el = exclusion_list(ER_0787a_1, Blank_data_df)
ER_0787a_1_rr = remove_rows(ER_0787a_1, ER_0787a_1_el)

ER_0787b_1_el = exclusion_list(ER_0787b_1, Blank_data_df)
ER_0787b_1_rr = remove_rows(ER_0787b_1, ER_0787b_1_el)

In [20]:
print(ER_0789a_1_rr.shape)
print(ER_0788a_1_rr.shape)
print(ER_0787a_1_rr.shape)
print(ER_0787b_1_rr.shape)

(86, 21)
(93, 21)
(65, 21)
(65, 21)


In [21]:
print(ER_0789a_1_el)
print(ER_0788a_1_el)
print(ER_0787a_1_el)
print(ER_0787b_1_el)

[1, 26, 29, 30, 43, 47, 49, 50, 51, 52, 53, 55, 59, 60, 61, 64, 69, 71, 72, 73, 74, 76, 77, 78, 79, 80, 83, 87, 88, 90, 91, 93, 94, 95, 98, 99, 100, 101, 102, 104, 105, 106, 109, 111, 114, 115, 119, 121, 122, 123, 124, 125, 128, 131, 132, 133, 134, 135, 139, 142, 145]
[1, 8, 9, 11, 15, 30, 46, 47, 50, 53, 54, 57, 60, 62, 63, 65, 66, 68, 74, 75, 76, 77, 78, 80, 83, 84, 85, 86, 89, 90, 91, 92, 94, 97, 98, 99, 101, 102, 103, 104, 105, 108, 109, 113, 114, 116, 118, 119, 121, 123, 124, 127, 128, 129, 130, 132, 133, 134, 135, 136, 137, 138, 139, 142, 143, 144, 145, 146, 148, 150, 151, 153, 156, 157, 159, 160, 168, 169, 171, 172, 173, 174]
[24, 32, 33, 35, 36, 37, 39, 41, 42, 46, 47, 49, 51, 55, 56, 57, 60, 65, 66, 68, 69, 71, 73, 74, 75, 77, 79, 81, 83, 84, 86, 89, 90, 94, 95, 97, 99, 101, 103]
[24, 32, 33, 35, 36, 37, 39, 41, 42, 46, 47, 49, 51, 55, 56, 57, 60, 65, 66, 68, 69, 71, 73, 74, 75, 77, 79, 81, 83, 84, 86, 89, 90, 94, 95, 97, 99, 101, 103]


In [22]:
ER_0789a_1_filtered = filter_df(ER_0789a_1_rr)
ER_0788a_1_filtered = filter_df(ER_0788a_1_rr)
ER_0787a_1_filtered = filter_df(ER_0787a_1_rr)
ER_0787b_1_filtered = filter_df(ER_0787b_1_rr)

In [23]:
print(ER_0789a_1_filtered.shape)
print(ER_0788a_1_filtered.shape)
print(ER_0787a_1_filtered.shape)
print(ER_0787b_1_filtered.shape)

(25, 21)
(19, 21)
(21, 21)
(21, 21)


In [24]:
ER_0789a_1_filtered.to_csv('/content/drive/My Drive/Reusser_alkene_screening/ER_0789a_1_filtered.csv')
ER_0788a_1_filtered.to_csv('/content/drive/My Drive/Reusser_alkene_screening/ER_0788a_1_filtered.csv')
ER_0787a_1_filtered.to_csv('/content/drive/My Drive/Reusser_alkene_screening/ER_0787a_1_filtered.csv')
ER_0787b_1_filtered.to_csv('/content/drive/My Drive/Reusser_alkene_screening/ER_0787b_1_filtered.csv')

In [25]:
ER_0789a_1_RIR = rm_no_RI(ER_0789a_1_filtered)
ER_0788a_1_RIR = rm_no_RI(ER_0788a_1_filtered)
ER_0787a_1_RIR = rm_no_RI(ER_0787a_1_filtered)
ER_0787b_1_RIR = rm_no_RI(ER_0787b_1_filtered)

In [26]:
print(ER_0789a_1_RIR.shape)
print(ER_0788a_1_RIR.shape)
print(ER_0787a_1_RIR.shape)
print(ER_0787b_1_RIR.shape)

(8, 21)
(6, 21)
(7, 21)
(7, 21)


In [27]:
ER_0789a_1_RIR.to_csv('/content/drive/My Drive/Reusser_alkene_screening/ER_0789a_1_RIR.csv')
ER_0788a_1_RIR.to_csv('/content/drive/My Drive/Reusser_alkene_screening/ER_0788a_1_RIR.csv')
ER_0787a_1_RIR.to_csv('/content/drive/My Drive/Reusser_alkene_screening/ER_0787a_1_RIR.csv')
ER_0787b_1_RIR.to_csv('/content/drive/My Drive/Reusser_alkene_screening/ER_0787b_1_RIR.csv')